In [88]:
import numpy as np

In [89]:
def v_next(v, row, column):
    if row < 0:
        row = 0
    elif row >= grid_size:
        row = grid_size - 1
    if column < 0:
        column = 0
    elif column >= grid_size:
        column = grid_size - 1
    return v[row, column]

def to_coordinates(i,j,action):
    assert isinstance(action, str)
    if action == 'up':
        row, column = 1, 0
    elif action == 'down':
        row, column = -1, 0
    elif action == 'left':
        row, column = 0, -1
    elif action == 'right':
        row, column = 0, 1
    else:
        raise ValueError('action is wrong!')
    return i+row, j+column

In [90]:
grid_size = 4
min_delta = 0.001
gamma = 1
reward = -1
actions = ['up','down','left','right']
policies = np.random.choice(actions, size=(grid_size,grid_size))
policies[0,0] = policies[grid_size-1, grid_size-1] = ''
v = np.zeros((grid_size, grid_size))
probs = [0.25]*4

In [91]:
## Iterative plice evaluation on page 75
while True:
    delta = 0
    new_v = v.copy()
    for i in range(grid_size):
        for j in range(grid_size):
            if not (i==j==0 or i==j==grid_size-1):
                new_v[i,j] = np.dot(probs, [v_next(v,i,j+1), v_next(v,i,j-1), v_next(v,i-1,j), v_next(v,i+1,j)]) + reward
                delta = max(delta, abs(new_v[i,j]-v[i,j]))
    v = new_v
    if delta < min_delta:
        break

v

array([[  0.        , -13.98945772, -19.98437823, -21.98251832],
       [-13.98945772, -17.98623815, -19.98448273, -19.98437823],
       [-19.98437823, -19.98448273, -17.98623815, -13.98945772],
       [-21.98251832, -19.98437823, -13.98945772,   0.        ]])

In [92]:
## Policy iteration on page 80
k = 1
policies = np.array([['','left','left','left'],
           ['down','left','left','left'],
           ['right','right','right','up'],
           ['right','right','right','']])     #avoid endless loop
v = np.zeros((grid_size, grid_size))

while True:
    ## plice evaluation
    while True:
        delta = 0
        new_v = v.copy()
        for i in range(grid_size):
            for j in range(grid_size):
                if not (i==j==0 or i==j==grid_size-1):
                    horizontal, vertical = to_coordinates(i,j,policies[i,j])
                    new_v[i,j] = v_next(v, horizontal, vertical)*gamma + reward
                    delta = max(delta, abs(new_v[i,j]-v[i,j]))
        v = new_v
        if delta < min_delta:
            break

    ## policy improvement
    policy_stable = True
    for i in range(grid_size):
        for j in range(grid_size):
            if not (i==j==0 or i==j==grid_size-1):
                idx = np.argmax([v_next(v,i+1,j),v_next(v,i-1,j),v_next(v,i,j-1),v_next(v,i,j+1)])
                if policies[i,j] != actions[idx]:
                    policy_stable = False
                policies[i,j] = actions[idx]
    
    k += 1
    if policy_stable:
        break
        
print('# of iteration:',k)
print(v)
print(policies)

# of iteration: 4
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
[['' 'left' 'left' 'up']
 ['down' 'down' 'up' 'up']
 ['down' 'up' 'up' 'up']
 ['down' 'right' 'right' '']]


In [96]:
## Value Iteration on page 83
v = np.zeros((grid_size, grid_size))
while True:
    delta = 0
    new_v = v.copy()
    for i in range(grid_size):
        for j in range(grid_size):
            if not (i==j==0 or i==j==grid_size-1):
                new_v[i,j] = max([v_next(v,i+1,j),v_next(v,i-1,j),v_next(v,i,j-1),v_next(v,i,j+1)])*gamma + reward
                delta = max(delta, abs(new_v[i,j]-v[i,j]))
    v = new_v
    if delta < min_delta:
        break

for i in range(grid_size):
    for j in range(grid_size):
        if not (i==j==0 or i==j==grid_size-1):
            idx = np.argmax([v_next(v,i+1,j),v_next(v,i-1,j),v_next(v,i,j-1),v_next(v,i,j+1)])
            policies[i,j] = actions[idx]
            
print(v)
print(policies)

[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
[['' 'left' 'left' 'up']
 ['down' 'down' 'up' 'up']
 ['down' 'up' 'up' 'up']
 ['down' 'right' 'right' '']]
